In [34]:
import boto3
import sagemaker
import sys
import os
import re
import numpy as np
import subprocess
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
import gzip
from io import BytesIO
import zipfile


In [20]:

# NOTE: S3 bucket name must begin with "deeplens-" for DeepLens deployment
bucket_name='dataset-car-images'
prefix = 'cars'

In [21]:
def setup_env():
    
    role = get_execution_role()

    sess = sagemaker.Session()

    
    AWS_REGION = 'us-east-1'
    s3 = boto3.resource('s3')

    s3_bucket = s3.Bucket(bucket_name)

    if s3_bucket.creation_date == None:
    # create S3 bucket because it does not exist yet
        print('Creating S3 bucket {}.'.format(bucket))
        resp = s3.create_bucket(
            ACL='private',
            Bucket=bucket
        )
    else:
        print('Bucket already exists')
    return role, sess, AWS_REGION, s3

role, sess, AWS_REGION, s3 = setup_env()

Bucket already exists


### Download Data and Create Manifest

Note this step 



In [37]:
def create_dataset_manifest(bucket_name, s3_bucket):
#     bucket = bucket_name
#     gzipped_key = 'car-damage-dataset.tar.gz'
#     uncompressed_key = 'car-damage-dataset/'
    
#     s3 = boto3.client('s3')

#     s3.upload_fileobj(                      # upload a new obj to s3
#         Fileobj=gzip.GzipFile(              # read in the output of gzip -d
#             None,                           # just return output as BytesIO
#             'rb',                           # read binary
#             fileobj=BytesIO(s3.get_object(Bucket=bucket, Key=gzipped_key)['Body'].read())),
#         Bucket=bucket,                      # target bucket, writing to
#         Key=uncompressed_key)   


    bucket = bucket_name

    s3 = boto3.client('s3', use_ssl=False)
    Key_unzip = 'car-damage-dataset/'

    prefix      = ""
    zipped_keys =  s3.list_objects_v2(Bucket=bucket, Prefix=prefix, Delimiter = "/")
    file_list = []
    for key in zipped_keys['Contents']:
        file_list.append(key['Key'])
        print(key)
    #This will give you list of files in the folder you mentioned as prefix
    s3_resource = boto3.resource('s3')
    #Now create zip object one by one, this below is for 1st file in file_list
    zip_obj = s3_resource.Object(bucket_name=bucket, key=file_list[0])
    print (zip_obj)
    buffer = BytesIO(zip_obj.get()["Body"].read())
    z = zipfile.ZipFile(buffer)
    for filename in z.namelist():
        file_info = z.getinfo(filename)
        s3_resource.meta.client.upload_fileobj(
            z.open(filename),
            Bucket=bucket,
            Key=Key_unzip + f'{filename}')
    
create_dataset_manifest(bucket_name, s3_bucket)

{'Key': 'car-damage-detection.zip', 'LastModified': datetime.datetime(2020, 1, 28, 8, 5, 52, tzinfo=tzlocal()), 'ETag': '"04995bbf494354af81b56d9352435db5-8"', 'Size': 128654048, 'StorageClass': 'STANDARD'}
s3.Object(bucket_name='dataset-car-images', key='car-damage-detection.zip')


In [14]:
def load_metadata(s3_bucket):
    
    if not os.path.exists('./car-damage-detection-manifest.zip'):
        s3_bucket.download_file('car-damage-detection-manifest.zip','./car-damage-detection-manifest.zip')
        print('Successfully Downloaded Image Training Data.')
    else:
        print('Data already exists')

load_data(s3_bucket)


Data already exists


In [16]:
def setup_classifier():

    training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")
    print (training_image)
    return training_image

training_image = setup_classifier()

811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:latest
